## Imports

In [26]:
import os
import rarfile
import requests
import pandas as pd
from typing import List
from io import StringIO 
from datetime import date

from bs4 import BeautifulSoup
from jita.settings.credentials import (
    qqp,
    datos_abiertos, 
    gasolina_hmo,
    casa_ley
)

year = str(date.today().year)

### qqp

In [2]:
page = requests.get(qqp)
soup = BeautifulSoup(page.content, 'lxml')
download_link = None
for index, link in enumerate(soup.find_all('a')):
    if year in link.text:
        download_link = link['href']
        break
    
if download_link:
    url = os.path.join(datos_abiertos, download_link)
    rar_response = requests.get(url)
    rar_response.raise_for_status()

    with open('temp.rar', 'wb') as f:
        f.write(rar_response.content)

    with rarfile.RarFile('temp.rar') as rf:
        csv_name = rf.namelist()[-2] 
        print(f"Extrayendo y leyendo: {csv_name}")

        with rf.open(csv_name) as csv_file_in_rar:
            df_qqp = pd.read_csv(csv_file_in_rar, encoding='utf-8', header=None)

os.remove('temp.rar')

Extrayendo y leyendo: 2025/08-2025_02.csv


In [6]:
nombres_columnas = [
    'PRODUCTO',
    'PRESENTACION',
    'MARCA',
    'CATEGORIA',
    'CATALOGO',
    'PRECIO',
    'FECHAREGISTRO',
    'CADENACOMERCIAL',
    'GIRO',
    'NOMBRECOMERCIAL', # Cambiado de 'NOMBRE_SUCURSAL' para coincidir con tu lista
    'DIRECCION',
    'ESTADO', # Movido para coincidir con el orden
    'MUNICIPIO', # Movido para coincidir con el orden
    'LATITUD',
    'LONGITUD'
]

df_qqp.columns = nombres_columnas

In [12]:
df_qqp[df_qqp['ESTADO'] == 'SONORA'].tail()

,PRODUCTO,PRESENTACION,MARCA,CATEGORIA,CATALOGO,PRECIO,FECHAREGISTRO,CADENACOMERCIAL,GIRO,NOMBRECOMERCIAL,DIRECCION,ESTADO,MUNICIPIO,LATITUD,LONGITUD
389178,PLÁTANO,1 KG. GRANEL. TABASCO/CHIAPAS/ROATÁN/PORTALIMÓN,S/M,FRUTAS FRESCAS,PACIC,23.9,2025-08-29,WAL-MART,SUPERMERCADO / TIENDA DE AUTOSERVICIO,WALMART SUCURSAL HERMOSILLO,"PASEO RÍO SONORA NO. 37A SUR, ESQ. BLVD. SOLID...",SONORA,HERMOSILLO,29.066837,-110.967247
389179,SARDINA,LATA 425 GR. EN TOMATE,AURRERA,PESCADOS Y MARISCOS EN CONSERVA,PACIC,18.0,2025-08-29,WAL-MART,SUPERMERCADO / TIENDA DE AUTOSERVICIO,WALMART SUCURSAL HERMOSILLO,"PASEO RÍO SONORA NO. 37A SUR, ESQ. BLVD. SOLID...",SONORA,HERMOSILLO,29.066837,-110.967247
389180,SARDINA,LATA 425 GR. EN TOMATE,GUAYMEX,PESCADOS Y MARISCOS EN CONSERVA,PACIC,43.0,2025-08-29,WAL-MART,SUPERMERCADO / TIENDA DE AUTOSERVICIO,WALMART SUCURSAL HERMOSILLO,"PASEO RÍO SONORA NO. 37A SUR, ESQ. BLVD. SOLID...",SONORA,HERMOSILLO,29.066837,-110.967247
389181,TORTILLA DE MAÍZ,1 KG. GRANEL,S/M,TORTILLAS Y DERIVADOS DEL MAIZ,PACIC,14.5,2025-08-29,WAL-MART,SUPERMERCADO / TIENDA DE AUTOSERVICIO,WALMART SUCURSAL HERMOSILLO,"PASEO RÍO SONORA NO. 37A SUR, ESQ. BLVD. SOLID...",SONORA,HERMOSILLO,29.066837,-110.967247
389182,ZANAHORIA,1 KG. GRANEL. MEDIANA,S/M,HORTALIZAS FRESCAS,PACIC,14.9,2025-08-29,WAL-MART,SUPERMERCADO / TIENDA DE AUTOSERVICIO,WALMART SUCURSAL HERMOSILLO,"PASEO RÍO SONORA NO. 37A SUR, ESQ. BLVD. SOLID...",SONORA,HERMOSILLO,29.066837,-110.967247


### gasolina

In [13]:
page = requests.get(gasolina_hmo)
soup = BeautifulSoup(page.content, 'lxml')
rows = []

for tr in soup.find_all('tr'):
    tds = tr.find_all('td', attrs={'data-label': True})
    if not tds:
        continue
    row = {td['data-label']: td.get_text(strip=True) for td in tds}
    rows.append(row)

In [15]:
pd.DataFrame(rows)

,Gasolinera,Dirección,Magna,Premium,Diesel
0,CM COMBUSTIBLES S.A. DE C.V.,Boulevard Ignacio Salazar Esquina Calle de Las...,21.99,24.99,
1,"GRUPO GASOLINERO LM, S.A. DE C.V.",Carretera a Nogales Km 1.7,22.99,25.49,22.36
2,ALMA DELIA MILLAN LOPEZ,Boulevard Antonio Quiroga No. 86,23.39,25.99,
3,GASOLINERA LA VERBENA SA DE CV,Boulevard Camino del Seri No. 356,22.19,25.49,
4,AUTOSERVICIO PALMIRA SA DE CV,Carretera a Nogales Km 5.5,22.49,24.99,24.49
...,...,...,...,...,...
144,ESTACION DE SERVICIO BACHOCO SA DE CV,Boulevard José María Morelos No. 329,22.69,25.8,
145,GASERVICIO EL LLANITO QUIROGA SA DE CV,Calle Dr Antonio Quiroga No. 135,21.99,25.49,24.69
146,"ESTACION PIRU, S.A. DE C.V.",Boulevard Camino del Seri S/N,21.39,24.99,
147,GASOLINERA RIO SONORA S.A. DE C.V.,Avenida Paseo Rio Sonora Norte S/N,21.59,25.49,24.69


### Casa Ley

#### selenium

In [24]:
import os
import re
import requests
import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# --- 1. Configuración (Respetando tus variables) ---
# Importa tus variables de configuración como lo tenías
from jita.settings.config import CASA_LEY_DATA
# Asegúrate de que tu archivo credentials.py o similar defina esta variable
from jita.settings.credentials import casa_ley 

# Constantes y preparación
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
CARPETA_SALIDA = CASA_LEY_DATA # Usamos tu variable para la carpeta

# --- 2. Función principal para organizar el código ---
def descargar_folleto_ley():
    """
    Función principal que encapsula toda la lógica de scraping y descarga.
    """

    urls_folleto = set()
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)

    try:
        # --- Extracción con Selenium ---
        driver.get(casa_ley) # Usamos la variable importada
        wait = WebDriverWait(driver, 20)
        print("⏳ Esperando que el iframe del folleto se cargue...")

        iframe = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'iframe[src*="publitas.com"]')))
        driver.switch_to.frame(iframe)
        print("✅ Iframe encontrado. Accediendo al folleto...")

        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "left")))
        print("✅ Folleto inicial cargado.")

        while True:
            # Guardar referencia a la imagen actual para la espera inteligente
            try:
                imagen_actual = driver.find_element(By.CSS_SELECTOR, "img.left").get_attribute('src')
            except:
                imagen_actual = "" # Si no la encuentra, no hay problema

            soup = BeautifulSoup(driver.page_source, 'lxml')
            
            for img in soup.select('img.left, img.right'):
                url_baja = img.get('src')
                if url_baja and 'publitas' in url_baja:
                    url_alta = re.sub(r'-at\d+', '-at2400', url_baja)
                    if url_alta not in urls_folleto:
                        print(f"📄 Página encontrada: {url_alta}")
                        urls_folleto.add(url_alta)

            try:
                next_button = driver.find_element(By.ID, "next_slide")
                if 'disabled' in next_button.get_attribute('class'):
                    print("🔚 Botón 'Siguiente' deshabilitado. Fin del folleto.")
                    break
                
                print("▶️ Pasando a la siguiente página...")
                next_button.click()

                # --- MEJORA: ESPERA INTELIGENTE ---
                # En lugar de time.sleep(2), esperamos a que la imagen cambie.
                wait.until(
                    lambda d: d.find_element(By.CSS_SELECTOR, "img.left").get_attribute('src') != imagen_actual
                )

            except Exception:
                print("🔚 No se pudo encontrar o hacer clic en el botón 'Siguiente'. Terminando.")
                break
    finally:
        driver.quit()
        print("\nNavegador cerrado.")

    # --- Descarga de Imágenes ---
    lista_urls = sorted(list(urls_folleto))
    if lista_urls:
        print(f"\n--- Iniciando descarga de {len(lista_urls)} imágenes ---")
        for i, url in enumerate(lista_urls):
            try:
                timestamp = datetime.datetime.now().strftime("%Y%m%d")
                nombre_archivo = os.path.join(
                    CARPETA_SALIDA,
                    f"pagina_{i+1:02d}_{timestamp}.jpg"
                )
                response = requests.get(url, headers=HEADERS, timeout=30)
                response.raise_for_status()
                with open(nombre_archivo, 'wb') as f:
                    f.write(response.content)
                print(f"✅ Guardada: {nombre_archivo}")
            except requests.exceptions.RequestException as e:
                print(f"❌ Error al descargar {url}: {e}")
        print("\n🎉 ¡Proceso completado!")
    else:
        print("\nNo se encontraron URLs para descargar.")

# --- Ejecución del script ---
if __name__ == "__main__":
    descargar_folleto_ley()

⏳ Esperando que el iframe del folleto se cargue...
✅ Iframe encontrado. Accediendo al folleto...
✅ Folleto inicial cargado.
📄 Página encontrada: https://view.publitas.com/89081/1913052/pages/36f9e53f-f8d0-49c8-a87f-f7fe37eba318-at2400.jpg
📄 Página encontrada: https://view.publitas.com/89081/1913052/pages/7efa1b9e-0d3a-4f9e-bf8b-17e79c78b6fd-at2400.jpg
▶️ Pasando a la siguiente página...
🔚 No se pudo encontrar o hacer clic en el botón 'Siguiente'. Terminando.

Navegador cerrado.

--- Iniciando descarga de 2 imágenes ---
✅ Guardada: C:\Users\angel.merino\Documents\GitHub\jita\datos\casa_ley\pagina_01_20250930.jpg
✅ Guardada: C:\Users\angel.merino\Documents\GitHub\jita\datos\casa_ley\pagina_02_20250930.jpg

🎉 ¡Proceso completado!


#### bs4

In [10]:
from jita.settings.config import CASA_LEY_DATA

In [3]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

page = requests.get(casa_ley, headers=headers)

soup = BeautifulSoup(page.content, 'lxml')

imgs_tag : List[str] = []
for tag in soup.find_all('img', class_='attachment-full'):
    imgs_tag.append(tag['src'])

if imgs_tag:
    print(f"✅ Se encontraron {len(imgs_tag)} imágenes. Iniciando descarga...")
    for url in imgs_tag:
        try:
            # 1. Extraemos el nombre del archivo de la URL
            nombre_archivo = url.split('/')[-1]
            print(f"📥 Descargando: {nombre_archivo}")
            
            # 2. Hacemos la solicitud para obtener la imagen (¡usando headers!)
            response_img = requests.get(url, headers=headers)
            response_img.raise_for_status() # Verifica si hay errores en la descarga
            
            # 3. Guardamos la imagen en la carpeta 'folletos' con su nombre original
            ruta_guardado = os.path.join(CASA_LEY_DATA, nombre_archivo)
            with open(ruta_guardado, 'wb') as f:
                f.write(response_img.content)
            
            print(f"   ✅ Guardado como: {ruta_guardado}")

        except requests.exceptions.RequestException as e:
            print(f"   ❌ Error al descargar {url}: {e}")
            
    print("\n🎉 ¡Proceso completado!")
else:
    print("❌ No se encontraron imágenes con la clase 'attachment-full'.")

❌ No se encontraron imágenes con la clase 'attachment-full'.


#### Extracción de texto

In [27]:
system_prompt = f"""
Eres un asistente que SOLO responde con JSON válido y nada más. 
Si no puedes identificar la información, devuelve un JSON vacío con la estructura. 
NO des explicaciones, NO uses enlaces, NO texto fuera del JSON.
Debes hacer OCR del texto de la imagen, interpretar las ofertas DE IZQUIERDA A DERECHA EN Z.
Debes devolver exclusivamente un JSON válido con la siguiente estructura:

Estructura:
{{
  "productos": [
    {{
      "nombre": "Nombre del producto, marca, o categoría",
      "precio": "Precio numérico COMPLETO con moneda SI es que se menciona",
      "oferta": "ej. 2x1, 3x2, 2x$precio, etc.",
      "presentación": "Cantidad y unidad kg, g, ml, L, etc.",
      "limites": "límite de compra máxima o mínima que aplica en la oferta",
      "condiciones": "Condiciones especiales para aplicar la oferta"
    }}
  ],
  "vigencia": "Fecha de los precios y ofertas válidas",
  "sucursales": "Sucursales donde aplican",
  "detalles" : "Detalles que se mencionan de forma general, como términos y condiciones."
}}

Ejemplo de salida:
{{
  "productos":[
    {{
      "nombre":"Leche Lala",
      "precio": null,
      "oferta":"2x$45",
      "presentación":"1L",
      "limites":"Máximo 4 por cliente"
    }},
    {{
      "nombre":"Platanos",
      "precio": "$20",
      "oferta": null,
      "presentación":"1 kg",
      "limites": null
    }},
    {{
      "nombre":"Cósmeticos L'Oreal",
      "precio": null,
      "oferta":"40%",
      "presentación":"todos los cósmeticos",
      "limites": null,
      "condiciones": "excepto lineadores"
    }}
  ],
  "vigencia":"del 23 al 29 de Septiembre 2025",
  "sucursales":"Todas las sucursales Casa Ley Hermosillo",
  "detalles":"Válido hasta agotar existencias"
}}

Si el producto está incompleto o ilegible, no lo incluyas.
Si un dato no aparece en el folleto, deja el campo en null o "".
Devuelve SIEMPRE un JSON válido.

Hoy es {date.today().isoformat()}, así que usa esta fecha como referencia si es necesario.
"""


In [ ]:
import ollama
from PIL import Image
from jita.settings.config import CASA_LEY_DATA

#casa_ley_folleto = f"{CASA_LEY_DATA}/window_0_1000.jpg"  # tu imagen
casa_ley_folleto = f"{CASA_LEY_DATA}/pagina_01.jpg"  # tu imagen

##### gemma3:27b

In [6]:
def sliding_window_vertical(image_path, window_height=1000, overlap=100, path=None):
    """
    Recorre la imagen con una ventana vertical (window_height) con solape (overlap).
    """
    img = Image.open(image_path)
    width, height = img.size
    tiles = []
    step = window_height - overlap
    for top in range(0, height, step):
        bottom = min(top + window_height, height)
        tile = img.crop((0, top, width, bottom))
        filename = f"window_{top}_{bottom}.jpg"
        tile.save(f"{path}/{filename}", quality=100)
        tiles.append(f"{path}/{filename}")
        if bottom == height:
            break
    return tiles

In [7]:
tiles_vertical = sliding_window_vertical(casa_ley_folleto, path=CASA_LEY_DATA)

In [12]:
# No necesitas importar 'date' aquí, puedes pasar la fecha directamente.
from datetime import date

# La fecha se calcula una vez y se inserta en el f-string.
current_date = date.today().isoformat()

system_prompt = f"""
Eres un motor de extracción de datos de alta precisión.
Tu única misión es analizar imágenes de folletos de supermercados y convertirlas en un objeto JSON estructurado y válido.

Misión Principal
1.  Analiza la imagen: Procesa el contenido visual de manera metódica: de arriba hacia abajo y de izquierda a derecha.
2.  Asocia la información: Vincula correctamente cada precio, oferta y descripción con el producto más cercano. Infiere la información del contexto (ej. si el precio dice "/kg", la presentación es "1 kg").
3.  Genera el JSON: Construye un único objeto JSON que se adhiera estrictamente a la estructura y reglas definidas a continuación.

Estructura de Salida Obligatoria (JSON)
Devuelve EXCLUSIVAMENTE un objeto JSON con la siguiente estructura. No incluyas texto, explicaciones ni comentarios antes o después del JSON.

{{
  "productos": [
    {{
      "nombre": "string | null - Nombre específico del producto, incluyendo marca si es visible (ej. 'Limón con semilla', 'Queso Crema Philadelphia').",
      "precio": "string | null - El precio final por unidad. Si el precio es parte de una oferta (ej. 2x$99), este campo debe ser null.",
      "oferta": "string | null - La promoción aplicable. Usa formatos consistentes: '2x1', '3x2', '2x$99', '50%', '$10 de descuento'.",
      "presentacion": "string | null - La cantidad, peso o volumen del producto (ej. '1 kg', '900 g', '1 L', 'Caja con 10 tabletas').",
      "limites": "string | null - Límite de piezas o kilos por cliente (ej. 'Máximo 5 kg por cliente').",
      "condiciones": "string | null - Cualquier otra condición para que la oferta aplique (ej. 'En la compra de 1', 'Pagando con Tarjeta X')."
    }}
  ],
  "vigencia": "string | null - El periodo de validez exacto de las ofertas (ej. 'Del 23 al 29 de Septiembre 2025').",
  "sucursales": "string | null - Las tiendas o ciudades donde aplica la promoción (ej. 'Sucursales Casa Ley en Hermosillo').",
  "detalles": "string | null - Cualquier texto general, como 'Válido hasta agotar existencias' o 'Aplican restricciones'."
}}

Reglas Críticas
- SOLO JSON: Tu respuesta debe ser únicamente el objeto JSON. Sin excepciones.
- INTEGRIDAD DE DATOS: Si el nombre o precio de un producto está cortado, borroso o es ilegible, OMITE ese producto por completo de la lista.
- MANEJO DE NULOS: Si un campo específico (ej. "oferta") no se menciona para un producto, su valor DEBE ser `null`.
- NO ASUMIR: No inventes información que no esté explícitamente en la imagen.

Contexto
- Fecha de hoy: {current_date}. Usa esta fecha como referencia para entender la vigencia del folleto, pero no la incluyas en la salida.

Ejemplo de Salida:
{{
  "productos": [
    {{
      "nombre": "Leche Lala 100 sin lactosa",
      "precio": null,
      "oferta": "2x$50",
      "presentacion": "1L",
      "limites": null,
      "condiciones": null
    }},
    {{
      "nombre": "Sandía Rayada",
      "precio": "$13.75",
      "oferta": null,
      "presentacion": "1 kg",
      "limites": null,
      "condiciones": null
    }},
    {{
      "nombre": "Todos los cosméticos L'Oreal",
      "precio": null,
      "oferta": "40% de descuento",
      "presentacion": null,
      "limites": null,
      "condiciones": "Excepto delineadores"
    }}
  ],
  "vigencia": "Vigencia del 23 al 29 de Septiembre 2025",
  "sucursales": "Tiendas Ley de Hermosillo",
  "detalles": "Válido hasta agotar existencias. Aclaraciones en tienda."
}}
"""

In [50]:
ofertas = []
for tile in tiles_vertical:
  resp = ollama.chat(
      model="gemma3:27b",
      messages=[
          {
              'role': 'system',
              'content': system_prompt
          },
          {
              'role': 'user',
              'content': "Folleto de casa ley",
              'images': [tile]  # 👈 lista de rutas de imagen
          }
      ],
      options={'temperature': 0}
  )
  ofertas.append(resp['message']['content'])

In [52]:
import json

for oferta in ofertas:
    # Quitamos el bloque ```json ... ```
    limpio = oferta.replace('```json', '').replace('```', '').strip()
    try:
        data = json.loads(limpio)
        print(json.dumps(data, indent=2, ensure_ascii=False))
    except json.JSONDecodeError:
        print("No es JSON válido:", limpio)
    print("-"*40)


{
  "productos": [
    {
      "nombre": "Pañal Huggies All Around",
      "precio": "$89.90",
      "oferta": null,
      "presentacion": "Paquete con 40 piezas",
      "limites": null,
      "condiciones": null
    },
    {
      "nombre": "Pañal Huggies Supreme",
      "precio": "$89.90",
      "oferta": null,
      "presentacion": "Paquete con 36 piezas",
      "limites": null,
      "condiciones": null
    },
    {
      "nombre": "Pañal Huggies Eco Protect",
      "precio": "$79.90",
      "oferta": null,
      "presentacion": "Paquete con 32 piezas",
      "limites": null,
      "condiciones": null
    },
    {
      "nombre": "Tostitos Salsa Verde y Habanero",
      "precio": "$29.90",
      "oferta": "3x2",
      "presentacion": "240 g",
      "limites": "Máximo 6 por cliente",
      "condiciones": null
    },
    {
      "nombre": "Atún Mazatun en agua o aceite",
      "precio": "$23.90",
      "oferta": null,
      "presentacion": "130 g",
      "limites": null,
      "condi

In [40]:
resp = ollama.chat(
      model="gemma3:27b",
      messages=[
          {
              'role': 'system',
              'content': system_prompt
          },
          {
              'role': 'user',
              'content': "Folleto de casa ley",
              'images': [casa_ley_folleto]  # 👈 lista de rutas de imagen
          }
      ],
      options={'temperature': 0}
  )

In [44]:
print(resp['message']['content'])

```json
{
  "productos": [
    {
      "nombre": "Shampoo Head & Shoulders",
      "precio": "$219.90",
      "oferta": null,
      "presentación": "650 ml",
      "limites": null
    },
    {
      "nombre": "Desodorante Axe",
      "precio": "$105",
      "oferta": null,
      "presentación": "115 g",
      "limites": null
    },
    {
      "nombre": "Ventilador",
      "precio": "$299.90",
      "oferta": null,
      "presentación": "16\"",
      "limites": null
    },
    {
      "nombre": "Aceites",
      "precio": "$239.90",
      "oferta": null,
      "presentación": "1 L",
      "limites": null
    },
    {
      "nombre": "Papel Higiénico",
      "precio": "$68.90",
      "oferta": null,
      "presentación": "32 rollos",
      "limites": null
    },
    {
      "nombre": "Salsa Catsup",
      "precio": "$29.90",
      "oferta": null,
      "presentación": "320 g",
      "limites": null
    },
    {
      "nombre": "Atún",
      "precio": "$22.90",
      "oferta": null,
     

##### llama3.2

In [ ]:
response = ollama.chat(
    model="llama3.2-vision:11b",
    messages=[
          {
              'role': 'system',
              'content': "extrae el contenido textual de las ofertas que veas en la imagen, procura que esté relacionada la promoción al producto"
          },
          {
              'role': 'user',
              'content': "Analiza este folleto",
              'images': [casa_ley_folleto]  # 👈 lista de rutas de imagen
          }
      ],
    options={'temperature': 0, 'format': 'json'}
  )

In [46]:
print(response['message']['content'])

El folleto de Ley es un documento que promueve la celebración de su aniversario con ofertas y descuentos en productos de la casa, limpieza, belleza, alimentación, bebidas, entre otros. El folleto se divide en secciones que presentan diferentes categorías de productos, como la sección de la casa, la sección de limpieza, la sección de belleza, la sección de alimentación, la sección de bebidas, la sección de productos para la familia, la sección de productos para la casa, la sección de productos para la limpieza, la sección de productos para la belleza, la sección de productos para la alimentación, la sección de productos para las bebidas, la sección de productos para la familia, la sección de productos para la casa, la sección de productos para la limpieza, la sección de productos para la belleza, la sección de productos para la alimentación, la sección de productos para las bebidas, la sección de productos para la familia, la sección de productos para la casa, la sección de productos pa

In [51]:
response = ollama.chat(
    model="llama3.2-vision:11b",
    messages=[
          {
              'role': 'system',
              'content': "qué ves en esta imagen?"
          },
          {
              'role': 'user',
              'content': "Analiza este folleto",
              'images': [casa_ley_folleto]  # 👈 lista de rutas de imagen
          }
      ],
    options={'temperature': 0, 'format': 'json'}
  )

In [52]:
casa_ley_folleto

'C:\\Users\\angel.merino\\Documents\\GitHub\\jita\\datos\\casa_ley/pagina_01.jpg'

In [53]:
print(response['message']['content'])

El folleto de la tienda de la competencia de la tienda de la competencia de la tienda de la competencia de la tienda de la competencia de la ti… (y así…)

El folleto de la tienda de la competencia de la ti… (y así…)

El folle… (y así…)

El… (y así…)

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El… (y…

El

In [59]:
casa_ley_folleto

'C:\\Users\\angel.merino\\Documents\\GitHub\\jita\\datos\\casa_ley/window_0_1000.jpg'

In [57]:
from ollama_ocr import OCRProcessor

# Initialize OCR processor
ocr = OCRProcessor(model_name='llama3.2-vision:11b')  # You can use any vision model available on Ollama

# Process an image
result = ocr.process_image(
    image_path=casa_ley_folleto,#image_path="path/to/your/pdf"
    format_type="json",  # Options: markdown, text, json, structured, key_value
    #custom_prompt=system_prompt # Optional custom prompt
)

Using default prompt: Extract all text from this image in en and format it as JSON, **strictly preserving** the structure.
                                - **Do not summarize, add, or modify any text.**
                                - Maintain hierarchical sections and subsections as they appear.
                                - Use keys that reflect the document's actual structure (e.g., "title", "body", "footer").
                                - Include all text, even if fragmented, blurry, or unclear.
                                


In [58]:
print(result)

**Title:** "La Tienda" (The Store)

**Subtitle:** "Versario" (Versary)

**Main Content:**

* **Header:** "La Tienda" (The Store)
* **Subtitle:** "Versario" (Versary)
* **Image:** A black-and-white illustration of a man and woman in a store, with a shopping cart and various products in the background.

**Footer:**

* **Logo:** "La Tienda" (The Store)
* **Contact Information:**
	+ Phone Number: 123-456-7890
	+ Address: 123 Main St, Anytown, USA
* **Social Media Links:**
	+ Facebook: @LaTienda
	+ Twitter: @LaTienda
	+ Instagram: @LaTienda
* **Copyright Information:** 2023 La Tienda. All rights reserved.

**Body:**

* **Product Section:**
	+ **Product 1:** "PANAL HUGGIES" (Huggies Panal)
		- **Price:** $9.99
		- **Description:** "All Around" (All Around)
		- **Size:** 40 pieces
	+ **Product 2:** "TOSTITOS" (Tostitos)
		- **Price:** $2.99
		- **Description:** "Sabor de la Venta" (Savor of the Sale)
		- **Size:** 2.9 oz
	+ **Product 3:** "DETERGENTE" (Detergent)
		- **Price:** $3.99
		- **De

In [5]:
from jita.settings.config import CASA_LEY_DATA
#casa_ley_folleto = f"{CASA_LEY_DATA}/pagina_01.jpg"  # tu imagen
casa_ley_folleto = f"{CASA_LEY_DATA}/window_0_1000.jpg"  # tu imagen


In [17]:
import base64
from io import BytesIO
from PIL import Image
from pathlib import Path

def encode_image_to_base64(image_path):
    """Convert an image file (string or Path) to base64 string."""
    path = Path(image_path)
    return base64.b64encode(path.read_bytes()).decode("utf-8")


In [18]:
encoded_image = encode_image_to_base64(casa_ley_folleto)

In [ ]:
import ollama


response = ollama.chat(
    model='llama3.2-vision:11b',
    messages=[
        {
            'role': 'user', 'system': system_prompt,
            'role': 'user', 'content': 'Ayúdame con este folleto',
            # For direct file path (Ollama handles reading the image):
            #'images': ['example.png']
            # If using base64 encoding:
            'images': [encoded_image]
        }
    ]
)
print(response['message']['content'])

##### qwen2.5-VL

In [31]:
response = ollama.chat(
    model="qwen2.5vl:7b",
    messages=[
          {
              'role': 'system',
              'content': "extrae el contenido textual de las ofertas que veas en la imagen, procura que esté relacionada la promoción al producto"
          },
          {
              'role': 'user',
              'content': "Analiza este folleto",
              'images': [casa_ley_folleto]  # 👈 lista de rutas de imagen
          }
      ],
    options={'temperature': 0, 'format': 'json'}
  )

In [ ]:
response = ollama.chat(
    model="qwen2.5vl:7b",
    messages=[
          {
              'role': 'system',
              'content': system_prompt,
              'role': 'user',
              'content': "Analiza este folleto",
              'images': [casa_ley_folleto]  # 👈 lista de rutas de imagen
          }
      ],
    options={'temperature': 0, 'format': 'json'}
  )

In [33]:
print(response['message']['content'])

Este folleto es un anuncio de una promoción de aniversario en un supermercado llamado "Ley". La promoción incluye una variedad de productos en diferentes categorías, como higiene y belleza, hogar y entretenimiento, ropa y baño, comida, bebidas, y productos para fiestas. Aquí hay una descripción general de las categorías y algunos productos destacados:

### Higiene y Belleza
- Shampoo
- Desodorantes
- Crema dental
- Tinte
- Crema corporal
- Jabón de tocador
- Perfumería fina

### Mejorando tu Hogar/Entretenimiento
- Pantallas
- Ventiladores
- Aceites y antifriajes
- Papel higiénico
- Lámparas
- Accesorios para pintura
- Pañales
- Bebés
- Macetas
- Muebles
- Utensilios para cocina
- Detergentes
- Productos para mascotas
- Limpieza del hogar
- Productos para la ropa

### Ropa / Baño y Recamara
- Estropajos
- Ropa interior y pijamas
- Papel higiénico
- Productos lácteos
- Atún
- Chiles
- Leche
- Harina
- Galletas
- Bebidas
- Servitallas
- Productos para la ropa

### Comida y algo más
- Sal

##### pytesseract

In [15]:
from jita.settings.config import CASA_LEY_DATA
casa_ley_folleto = f"{CASA_LEY_DATA}/pagina_01.jpg"  # tu imagen
#casa_ley_folleto = f"{CASA_LEY_DATA}/window_0_1000.jpg"  # tu imagen

In [11]:
from PIL import Image, ImageFilter, ImageOps
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Users\angel.merino\AppData\Local\Programs\Tesseract-OCR\tesseract.exe"

# Ruta de la imagen
image_path = casa_ley_folleto

# Abrir la imagen
img = Image.open(image_path)

# Preprocesamiento básico
img = img.convert("L")                 # Escala de grises
img = ImageOps.invert(img)             # Invertir si el texto es claro sobre fondo oscuro
img = img.point(lambda x: 0 if x < 140 else 255)  # Binarización simple

# OCR
text = pytesseract.image_to_string(img, lang="spa")

print("Texto extraído:")
print(text)


Texto extraído:
La Tradición
delos Mejores

Precios

Mejorando tu:H Ropa / Baño y Recámara: [Comidaly/algomas)
JO JO suimeoo (A a DESODORANTE e Todoslos LIQUIDACIÓN DE ROPA Todostos ESTROPAJOS ACEITE ATÚN Es acue PRODUCTO LÁCTEO ACEITE SS BEBIDA BEBIDA 4 REFRESCO
A A En Arrosol A. a ACEITES... Para Baño comestible A : Duopack Vegetal Minibrick Powerade” Coca Cola
2 Headá 7 : ACEITES EXTERIOR, INTERIOR ae : MERO p 3 inibrick ade co
Shoulders” OIL BARDAHL a. Y PIJAMAS De Blancos) eya Le so í Sabor Original
e > Y e

os A e y
Nutriolis a ce 1209 descomi Y Erianzana
de got WII f a

: Para toda ae y]
de, La Familia. rra (1 1322 /
3 ETA

O]

o PAPEL HIGIÉNICO

INPC ” Ultra Jumbo

PANTALLA : A
Pe pira sólo > con 162 iezas

SHAMPOD VENTILADOR * ACEITES Y PAPEL HIGIÉNICO SALSA CASERA - 11833
2 Acondicionador d-dot 'ANTICONGELANTES Kleenex' Brand Rojo Mexicona A en rajas BEBIDA SERVITOALLA
sure Pe Evive m MOTOR -- ns ennez HERDEZ La Costeña ¡RauPar Bro
a tae sl PRO Cro QUO ¿a de 18 E MiBibrick al

In [12]:
import cv2
from PIL import Image
import pytesseract

# Leer con OpenCV
img = cv2.imread(image_path)

# Convertir a escala de grises
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Binarización (umbral adaptativo)
thresh = cv2.adaptiveThreshold(
    gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2
)

# Guardar o pasar directamente a PIL
pil_img = Image.fromarray(thresh)

# OCR
text = pytesseract.image_to_string(pil_img, lang="spa")

print("Texto extraído:")
print(text)


Texto extraído:
5), La Tradición ===
4 Badilla
"E “Precios :

rra] es y Belleza wma P Mejorando tu Hogar/Entretenimiento

Ropa / Baño y Recamara amas Comiday/al Y 26 (ES

SHAMPOO a DESODORANTE Todoslos + e LIQUIDACIÓN DE ROPA Lat los ESTROPAJOS ACEITE ATÚN En agua: PRODUCTO LÁCTEO ACEITE BEBIDA REFRESCO
y Aerosol AXES ACEITES EXTERIOR, INTERIOR: (| a rfi Z Comestible eenacelte Duopack z Vegetal Minibrick : A... Coca Cola9
de £89-979, Doy Pack Ú e pseto de soya Leyo Jue k Sabor Original
dos mscay :BARDAHLO A 7 Y PIJAMAS od Él ES 00 Nutriolis de 800m!, del
Es CRES E a Ya Hasta pane , A A
(E) de 459-509 y AT Py (0) z PAPEL MIGIÉNICO ,
Eto 50 PANTALLA grrr ; 2 Pétalos :
Except Pieza a sólo e conté piezas [aso pa
Clinica kee! FullHD/A % 900" 0 A , pe 12, 69% x 4 90
Fragancia =
2160" y dea 437 $ y -

SHAMPODo Acond. " SHAMPOO * VENTILADOR PAPEL HIGIÉNICO PAPEL HIGIÉNICO SALSA CASERA CHILES Es LECHE py PARA,
e o Ca a ETA is pa e a a a
ATA Toqueccdal ; Pieza de 680mt_ . 1 = Eli Ena piezas de 2

In [22]:
from paddleocr import PaddleOCR
from PIL import Image
import numpy as np

# Crear OCR con parámetros actualizados
ocr = PaddleOCR(
    lang="es",
    use_textline_orientation=True,
    text_det_box_thresh=0.5
)

# Abrir imagen y convertir a array
image = Image.open(casa_ley_folleto).convert('RGB')
image_np = np.array(image)

# Ejecutar OCR
result = ocr.predict(image_np)

# Procesar y mostrar resultados
for res in result:
    if isinstance(res, list):
        for line in res:
            if isinstance(line[1], list) and len(line[1]) == 2:
                text, conf = line[1]
                print(f"Text: {text}, Confidence: {conf:.2f}, Bounding Box: {line[0]}")
            else:
                print(f"Warning: Unexpected format in line: {line}")
    else:
        print(f"Warning: Unexpected result format: {res}")


Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\angel.merino\.paddlex\official_models\PP-LCNet_x1_0_doc_ori`.
Creating model: ('UVDoc', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\angel.merino\.paddlex\official_models\UVDoc`.
Creating model: ('PP-LCNet_x1_0_textline_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\angel.merino\.paddlex\official_models\PP-LCNet_x1_0_textline_ori`.
Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\angel.merino\.paddlex\official_models\PP-OCRv5_server_det`.
Creating model: ('latin_PP-OCRv5_mobile_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manu

        ...,
        [253, ..., 129]],

       ...,

       [[237, ...,  36],
        ...,
        [234, ...,  35]]], shape=(1774, 3247, 3), dtype=uint8), 'model_settings': {'use_doc_orientation_classify': True, 'use_doc_unwarping': True}, 'angle': 0, 'rot_img': array([[[244, ..., 135],
        ...,
        [253, ..., 129]],

       ...,

       [[237, ...,  36],
        ...,
        [234, ...,  35]]], shape=(1774, 3247, 3), dtype=uint8), 'output_img': array([[[238, ..., 139],
        ...,
        [252, ..., 177]],

       ...,

       [[249, ..., 250],
        ...,
        [219, ...,  38]]], shape=(1774, 3247, 3), dtype=uint8)}, 'dt_polys': [array([[600, 481],
       ...,
       [600, 544]], shape=(4, 2), dtype=int16), array([[695, 488],
       ...,
       [695, 509]], shape=(4, 2), dtype=int16), array([[745, 488],
       ...,
       [745, 499]], shape=(4, 2), dtype=int16), array([[1994,  486],
       ...,
       [1992,  508]], shape=(4, 2), dtype=int16), array([[2327,  479],
       .